In [1]:
# 1 Import libraries + custom functions

import numpy as np
from utils import filename_from_path
from keras.models import Sequential
from keras.optimizers import *
from keras.regularizers import l2
from keras.layers import *
from keras.utils.np_utils import to_categorical
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
from skimage.io import imread
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
from keras.callbacks import ReduceLROnPlateau,EarlyStopping
import keras
import glob
import platform
from datetime import datetime
import pickle

def reshape_to_view_img(img):
    (x,y) = (img.shape[0],img.shape[1])
    return np.reshape(img,(x,-1,))

def filename_from_path(path):
    return path.split('/')[-1]

Using TensorFlow backend.


In [2]:
#verify GPU is up n running
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17321119024075214032
]


In [3]:
# 2 Load data + initialize variables

train_labels = pd.read_csv('train_onelabel.csv',index_col='image')
get_class_of = train_labels['class'].to_dict()                                             #dict image->class
get_images_of = train_labels.groupby('class').apply(lambda x: x.values.tolist()).to_dict() #dict ???

input_size = (64, 64)
output_size = len(list(train_labels['class'].unique()))

label_map = pd.read_csv('label_map.txt',header=None,names=['name','id'],sep=' ')
get_name_of_class = label_map['name'].to_dict()
''' get_name_of_class[9] >> 'ctenophore_cydippid_tentacles' '''
get_class_name_of = {v: k for k, v in get_name_of_class.items()}
''' get_class_name_of['ctenophore_cydippid_tentacles'] >> 9 '''

training_paths_list = glob.glob('competition_data/train_images/*')     #normal (not numpy) list of str  
test_set_list = glob.glob('competition_data/test_images/*')          #normal (not numpy) list of str 

# savedmodelpath = 'trainHistory/12-08 18.54, train_err=0.70, test_err=0.73double_kernel_size_termatismeno.h5'

np.random.seed(123)  # for reproducibility

#model parameters

batch_size=64
reg_lambda = 0.01/255
dropout_value=0.25

#handle imbalanced classes

#Read training data

class_counts = train_labels['class'].value_counts()
class_weights = 1/np.log(class_counts)
#class_weights = class_weights.to_dict()

In [4]:
# 3 Load & Preprocess input data

def inverse_grayscale(pixel_value):
    return (255.-pixel_value)
def inverse_scale_center(pixel_value):
    return (((255.-pixel_value)/255)-0.5)
v_inverse_grayscale = np.vectorize(inverse_grayscale)
v_inverse_scale_center = np.vectorize(inverse_scale_center)

input_size = (64,64)      # ! parameter
training_images = (load_img(p, target_size=input_size, grayscale=True) for p in training_paths_list)
traning_images_array = np.array([img_to_array(im) for im in training_images]) #numpy array of numpy arrays (pictures) (X_train)
del training_images

test_images = (load_img(p, target_size=input_size,grayscale=True) for p in test_set_list)
test_images_array = np.array([img_to_array(im) for im in test_images])
del test_images

if (platform.system()=='Windows'):
    all_labels = list(map(lambda p: get_class_of[p.split('\\')[-1]] , training_paths_list))
else:
    all_labels = list(map(lambda p: get_class_of[p.split('/')[-1]] , training_paths_list))

traning_images_array = v_inverse_scale_center(traning_images_array)   #apply inverse grayscale transformation before splitting to train+validation
test_images_array = v_inverse_scale_center(test_images_array)         #apply inverse grayscale transformation to test data as well

#We will take ALL images for the final training - normally we split 80-20 (train-validation)
# traning_images_array, validation_images_array, train_labels , validation_labels = train_test_split(traning_images_array, 
#                                                                                                    to_categorical(all_labels), 
#                                                                                                    stratify=to_categorical(all_labels), 
#                                                                                                    test_size=0.2, random_state=41)

#split training and cross validation data

traning_images_array = traning_images_array.astype('float32')
# validation_images_array = validation_images_array.astype('float32')
test_images_array = test_images_array.astype('float32')

In [6]:
# Image Augmentation
train_datagen = ImageDataGenerator(rotation_range=90,
                                   featurewise_center=True,
                                   featurewise_std_normalization=True,
                                    samplewise_std_normalization=False,
                                    samplewise_center=False,
                                   #rescale=1./255.,
                                   #zca_whitening=True,
                                   horizontal_flip=True,
                                   vertical_flip=False,
                                   zoom_range=0.1,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range = (20/360)*(2*np.pi)
                                )


# valid_datagen = ImageDataGenerator(
#             featurewise_center=True,
#             featurewise_std_normalization=True,
#             samplewise_std_normalization=False,
#             samplewise_center=False,
#         )


test_datagen = ImageDataGenerator(
            featurewise_center=True,
            featurewise_std_normalization=True,
            samplewise_std_normalization=False,
            samplewise_center=False,
        )


train_datagen.fit(traning_images_array)    #used for zca whitening/featurewise_center & normalization
# valid_datagen.fit(validation_images_array)
test_datagen.fit(test_images_array)

###
savepath = 'augmented_shit2//'

img = load_img('competition_data/train_images/419.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in train_datagen.flow(x, batch_size=1,
                          save_to_dir=savepath, save_prefix='newshit', save_format='jpeg'):
    i += 1
    if i > 20:
        break 